In [ ]:
# Introduction to Python (CIST 005A) Midterm Project
# First-grade Math Electronic Flash Cards: Math1Flash
# Author: Steve DeGrange
#
# Math1Flash is a simulation of a flash card educational activity.
#
# With physical flash cards, you have a stack of cards with a problem
# on the front side and the answer on the reverse side. A card is
# drawn and you are shown the front side for you to answer. If
# you don't get the answer right, you are shown the reverse side.
#
# In Math1Flash, you (supposedly a first-grader learning math) are
# given a random arithmetic or number compare problem to answer
# (a card is drawn from the stack of shuffled cards) and shown the
# problem (the front side). You are given up to three chances to
# answer correctly and then you are shown the answer (reverse side).
#
# Examples: What is 9 x 11?  answer a number     (arithmetic)
#           Is 10 > 16?      answer yes or no    (compare numbers)
#
# If you answer correctly, you are rewarded with praise.
#
# If you don't answer correctly, you are given up to two more
# tries (three total) to get it right and then shown the answer.
# Before each retry you're given a "hint" such as "That's too big"
# or "That's not yes or no".
#
# You can skip a problem/retry by just entering "" instead of an
# answer if you want to go to another question instead (or end).
#
# This is repeated (like drawing another card from the stack)
# until you reply negatively to wanting another problem question.
#
# In addition to entering "" to skip a problem, you can abbreviate
# "yes" and "no" down to one letter at all yes/no prompts and take
# the default of "yes" at a yes/no continuation-type prompt.
#
# Emojis and message coloring are used to make it more visually stimulating:
#      Arithmetic items are in blue.
#      Compare numbers items are in green.
#      Kudos output is partly in red.
#      Emojis strewn throughout

import random                                 # for randint and choice

# Define some constants
RED, GREEN, BLUE, BOLD, END = "\033[0;31m\033[1m","\033[0;32m\033[1m","\033[0;34m\033[1m", "\033[1m", "\033[0m"  # ANSI codes
ARITHCOLOR, COMPCOLOR = BLUE, GREEN           # for coloring some output specific to the type of operator
yesNo = ("yes", "ye", "y", "no", "n")         # Valid replies to yes/no prompts
arithmetic, compare = ("+", "-", "x"), ("=", "≠","<", ">", "≤", "≥")   # Supported operators
SALUTE, GRIN1, GRIN2, PARTY1, PARTY2 = "\U0001fae1", "\U0001f600", "\U0001f601", "\U0001f973", "\U0001f389"   # Emojis
XFINGERS, TROPHY, MEDAL, SAD, WORRY = "\U0001f91e", "\U0001f3c6", "\U0001f947", "\U0001f61e", "\U0001f61f"    # Emojis
HAND, THUMBUP, THUMBDOWN = "\U0000270b", "\U0001f44d", "\U0001f44e"                                          # Emojis

### Function getOperator returns a randomly chosen allowed arithmetic or compare numbers operator
def getOperator():
    return random.choice(arithmetic * 2 + compare)   # double up on arithmetic since compare has 2x more ops

### Function getOperands returns a list of two random whole numbers (aka non-negative integers) to use as operands
def getOperands(operator):
    maxOperand = 15 if operator == "x" else 25    # cut back for multiplication
    return [random.randint(0, maxOperand), random.randint(0, maxOperand)]

### Function getQuestion returns the current problem as a question, such as "What is 6 x 9?" or "Is 12 > 10?".  It is
### returned as a 4-item list to aid coloring of main q. [0] = whole q, [1] = prefix, [2] = main q, [3] = suffix.
def getQuestion(operator, operands):
    question = ["", "Is ", "%d %s %d" % (operands[0], operator, operands[1]), " ?   (answer yes or no, Enter=skip)"]  # init compare
    if operator in arithmetic:     # change form of question from "Is xxx? to "What is xxx?"
        question[1] = "What is "
        question[3] = " ?   (answer a number, Enter=skip)"
    question[0] = question[1] + question[2] + question[3]
    return question
    
### Function getAnswer returns the answer to the current problem, such as 54 for problem 9 x 6.
### The answer is returned as a 2-element list. [0] is the internal (int or True / False) form and [1] is the external (/ string form.
def getAnswer(operator, operands):
    answer = [None, ""]
    match operator:
        case "+":
            answer[0] = operands[0] + operands[1]
        case "-":
            answer[0] = operands[0] - operands[1]
        case "x":
            answer[0] = operands[0] * operands[1]
        case "=":
            answer[0] = operands[0] == operands[1]
        case "≠":
            answer[0] = operands[0] != operands[1]
        case "<":
            answer[0] = operands[0] < operands[1]
        case ">":
            answer[0] = operands[0] > operands[1]
        case "≤":
            answer[0] = operands[0] <= operands[1]
        case "≥":
            answer[0] = operands[0] >= operands[1]
        case _:
            return answer
    if operator in arithmetic:
        answer[1] = str(answer[0])
    else:
        answer[1] = "yes" if answer[0] else "no"
    return answer
   
### Function getProblem returns a list with the question / answer given the operator / operands
def getProblem(operator, operands):
    question = getQuestion(operator, operands)
    answer = getAnswer(operator, operands)
    return [question, answer]

### Function getHint returns text for hinting why an incorrect answer was wrong
def getHint(answerUser, answer, operator):
    if operator in compare:
        return "not yes or no" if answerUser not in yesNo else "wrong, but you'll get it"
    if not answerUser.isdigit():
        return "not a whole number"
    return "not big enough" if int(answerUser) < answer[0] else "too big"

### Function validateAnswer returns True or False meaning the answer from the user is correct or not
def validateAnswer(answerUser, answerCorrect, operator):
    if answerUser == "":
        return False
    if operator in arithmetic:
        return True if answerUser.isdigit() and int(answerUser) == answerCorrect[0] else False
    return True if answerCorrect[1].startswith(answerUser) else False

### Function getYesNo asks for "yes" or "no" (or "" for default) or a shortening like "n" or "ye".
### It returns True if the reply was yes and False if it was no.
def getYesNo(prompt, default = "yes"):
    reply = input(prompt).lower()
    if reply == "":
        reply = default
    while reply not in yesNo:
        reply = input("Please say yes or no. Try again.\n ").lower()
        if reply == "":
            reply = default
    return True if reply[0] == "y" else False

#-----------------------#
#     mainline code     #
#-----------------------#
# Game initialization
attempts, correct = 0, 0
user = input("My name is Flash.  What's yours? ")
if user == "":
    user = "math student"
print(f"I'm pleased to meet you, {BOLD}{user}{END}! {HAND}\n")
print(f"It's time for {BOLD}math questions{END}!")
print(f"Some are {ARITHCOLOR}arithmetic{END} and some {COMPCOLOR}compare numbers{END}.")

moreMath = getYesNo(f"\nAre you ready to start? {THUMBUP} {THUMBDOWN}\n   Say yes or no.   (Enter=yes)\n  ")
# Present new problems until they answer no to whether they want a new problem
while moreMath:
    # Build the next question and answer
    operator = getOperator()                               # get a new operator
    if operator in arithmetic:
        opType, opColor = "aritmetic", ARITHCOLOR
    else:
        opType, opColor = "compare numbers", COMPCOLOR
    operands = getOperands(operator)                       # get two new operands
    question, answer = getProblem(operator, operands)      # generate new problem and answer

    # If multiplication and the answer is big, or subtraction and the answer is negative, loop
    #   getting new operands and problem, keeping same operator, until the problem is easier
    while (operator == 'x' and answer[0] > 100) or (operator == '-' and answer[0] < 0):
        operands = getOperands(operator)                   # get another pair of operands
        question, answer = getProblem(operator, operands)  # regenerate the problem and answer
    
    # Present the question to the user and ask for the answer, repeating up to twice more as needed
    # A reply of "" anywhere indicates skip this question and get another (or end).
    print(f"\nTry this {opColor}{opType}{END} problem:\n   {question[1]}{opColor}{question[2]}{END}{question[3]}")
    reply = input("   ").strip().lower()

    # Validate answer provided, skipping problem if answer is "", and retry up to two more times
    if reply != "":                        # "" on 1st attemptl; they haven't answer wrong so don't count attempt
        attempts += 1
        # See if they got it right on the first answer and if so give them kudos
        if validateAnswer(reply, answer, operator):
            correct += 1
            # Pick a random kudos praising phrase so its not too boring
            match random.randint(0, 4):
                case 0:
                    print(f"\n{GRIN1} {RED}Right!{END} You're really super at this {BOLD}{user}{END}!")
                case 1:
                    print(f"\n{PARTY1} {RED}Correct!{END} You're doing great, {BOLD}{user}{END}!")
                case 2:
                    print(f"\n{GRIN1} {RED}You got it, {user}!{END} You're doing really good{END}.")
                case 3:
                    print(f"\n{PARTY2} {RED}Nice job, {user}!{END} You're super at this!")
                case _:
                    print(f"\n{GRIN2} {RED}That's right!{END} Keep it up, {BOLD}{user}.{END}")
        else:   # Second chance
            reply = input(f"\n{WORRY} That's {getHint(reply, answer, operator)}. Try again! {XFINGERS}\n   ").strip().lower()
            if reply != "":                                 # "" on 1st attempt skips the whole question
                if validateAnswer(reply, answer, operator):
                    correct += 1           # We're easy graders :)  Count it as ok since they got it 2nd time
                    print(f"\n{GRIN2} {RED}Correct!{END} You got it, {BOLD}{user}{END}! {THUMBUP}")
                else:   # Third and final chance
                    reply = input(f"\n{SAD} That's {getHint(reply, answer, operator)}. Try once more. {XFINGERS}\n   ").strip().lower()
                    if reply != "":                           # "" on 1st attempt skips the whole question
                        if validateAnswer(reply, answer, operator):
                            correct += 1   # We're really easy graders :)  Count it as ok since they finally got it
                            print(f"{GRIN1} {RED}Yes, you did it!!{END} Good job, {BOLD}{user}{END}! {THUMBUP}")
                        else:
                            # Incorrect all three times.  Time to punt. Just tell them the right answer and move on.
                            print(f"\n{THUMBDOWN} That's not right, either. It was hard.\nThe answer was {opColor}{answer[1]}{END}.")

    if reply == "":  # Problem skipped by user answering "". Tell them the correct answer before moving on.
        print(f"OK. The answer was {opColor}{answer[1]}{END}.") 
    
    # See if they want to get a new math problem or just to quit
    moreMath = getYesNo(f"\nDo you want a new problem? {THUMBUP} {THUMBDOWN}\n   Say yes or no.   (Enter=yes)\n  ")

# Time to go. Print goodbye messages.
if attempts > 1 and correct > attempts / 2:
    if correct == attempts:
        print(f"{MEDAL} {RED}You didn't miss any! {correct} out of {attempts}!{END} Most excellent!{TROPHY}")
    else:
        print(f"{RED}You got {correct} out of {attempts} right!{END} Nice job!")

print(f"Bye, {BOLD}{user}{END}. Come back soon. {HAND}")
